<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/CatOrDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import glob 
import os.path as osp 
import random 
import numpy as np 
import json 
from PIL import Image 
from tqdm import tqdm 
import matplotlib.pyplot as plt 
%matplotlib inline 

import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.utils.data as data 
import torchvision 
from torchvision import models , transforms

In [0]:
class ImageTransform():


    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        
        return self.data_transform[phase](img)

In [15]:
def make_datapath_list(phase='train'):
  rootpath = '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/'
  target_path = osp.join(rootpath+phase+'/**/*.jpg')
  print(target_path)
  
  path_list = []
  
  #globを利用してサブディレクトリまでパスを取得する
  for path in glob.glob(target_path):
    path_list.append(path)
  
  return path_list

#実行
train_list = make_datapath_list(phase='train')
test_list = make_datapath_list(phase='test')

train_list

/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/**/*.jpg
/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/test/**/*.jpg


['/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/2671108251.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/26663494501.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/2685832857.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/26765080749.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/26569790555.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/26511709600.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/25801648582.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/25857232738.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/2541127345.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/2592564716.jpg',
 '/content/drive/My Drive/Cat_and_Dog/dogs-vs-cats/dogs-vs-cats/train/cat/2504364501

In [23]:
size = 224 
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)

#Datasetの作成

class DogCatDataset(data.Dataset):
  """
  犬猫の画像のDatasetクラス.PytorchのDatasetクラスを継承
  """
  def __init__(self,file_list,transform=None,phase='train'):
    self.file_list = file_list
    self.transform = transform 
    self.phase = phase
    
  def __len__(self):
    """画像の枚数を返す"""
    return len(self.file_list)
  
  def __getitem__(self,index):
    '''
    前処理をした画像のTensoor形式のデータとラベルを取得
    '''
    img_path = self.file_list[index]
    img = Image.open(img_path)
    
    #画像の前処理を実施
    img_transformed = self.transform(
        img,self.phase
    ) #torch.Size([3,224,224])
    
    #画像のラベルをファイル名から抜き出す
    if self.phase == 'train':
      label = img_path[62:71]
    elif self.phase == '':
      label = img_path[62:71]   ###ここらが変
    
    #ラベルを数値に変換
    if label == 'cat':
      label = 0
    elif label == 'dog':
      label = 1
      
    return img_transformed, label
  
 #実行
train_dataset = DogCatDataset(
      file_list = train_list,transform=ImageTransform(size,mean,std),phase='train'
    )
test_dataset = DogCatDataset(
      file_list = test_list,transform=ImageTransform(size,mean,std),phase='test'
    )
    
#indexの確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
train/cat
